In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import obb_login
from openbb import obb
from humbldata.core.utils.env import Env
import polars as pl

# obb_login()
obb.account.login(pat=Env().OBB_PAT, remember_me=True)

In [ ]:

from humbldata.toolbox.technical.mandelbrot_channel.model import (
    calc_mandelbrot_channel,
    calc_mandelbrot_channel_historical,
)


data = (
    obb.equity.price.historical(
        ["AAPL"],
        provider="yfinance",
        start_date="2020-01-01",
        end_date="2024-01-01",
        adjustment="splits_and_dividends",
    ).to_polars()
).drop(["dividends", "stock_splits"]).with_columns(pl.lit("AAPL").alias("symbol"))

mandelbrot = calc_mandelbrot_channel_historical(  # noqa: ERA001
        data,
        window="1m",
        rv_adjustment=True,
        rv_method="std",
        rv_grouped_mean=False,
        rs_method="RS",
        live_price=False,
    ).collect()
mandelbrot

# Setting up UserTable

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.standard_models.portfolio.analytics.user_table import UserTableQueryParams
from humbldata.core.utils.openbb_helpers import get_latest_price


symbols = UserTableQueryParams(symbol="AAPL, MSFT, NVDA").symbol
symbols



get_latest_price(symbols)

# Tesing Async


## Get Sector Async

In [27]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import aget_etf_sector, aget_equity_sector
import polars as pl

stocks = pl.Series([
    "GOOGL",  # Communication Services
    "AMZN",   # Consumer Discretionary
    "KO",     # Consumer Staples
    "XOM",    # Energy
    "JPM",    # Financials
    "JNJ",    # Health Care
    "HON",    # Industrials
    "AAPL",   # Information Technology
    "LIN",    # Materials
    "PLD",    # Real Estate
    "NEE"     # Utilities
])
# test = (await aget_etf_sector(symbols=stocks, provider="yfinance")).collect()
test = (await aget_etf_sector(symbols=["AAPL", "XLE"], provider="yfinance")).collect()
# test2 = (await aget_equity_sector(symbols=stocks, provider="yfinance")).collect()
# test2 = (await aget_equity_sector(symbols=["XLE", "AAPL"], provider="yfinance")).collect()
test

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


symbol,sector
str,str
"""XLE""","""Equity Energy"""


In [20]:
# obb.etf.info(symbol=['AAPL', "XLE"], provider='yfinance').to_polars()
obb.etf.info(symbol=['AAPL', "NVDA"], provider='yfinance').to_polars()

OpenBBError: Results not found.

In [ ]:


async def aget_etf_sector(
    symbols: str | list[str] | pl.Series,
    provider: "yfinance" | "openbb" = "yfinance",
) -> pl.LazyFrame:
    """
    Asynchronously retrieves the sector information for the given ETF symbol(s).
    """
    loop = asyncio.get_event_loop()

    # Ensure symbols is a list
    if isinstance(symbols, str):
        symbols = [symbols]
    elif isinstance(symbols, pl.Series):
        symbols = symbols.to_list()

    try:
        result = await loop.run_in_executor(
            None, lambda: obb.etf.info(symbols, provider=provider)
        )

        df = (
            result.to_polars()
            .lazy()
            .select(["symbol", "category"])
            .rename({"category": "sector"})
        )

        # Create a LazyFrame with all input symbols
        all_symbols = pl.LazyFrame({"symbol": symbols})

        # Left join to include all input symbols, filling missing sectors with null
        return (
            all_symbols
            .join(df, on="symbol", how="left")
            .with_columns([
                pl.when(pl.col("sector").is_null())
                .then(None)
                .otherwise(pl.col("sector"))
                .alias("sector")
            ])
        )

    except Exception as e:
        # If an OpenBBError occurs, return a LazyFrame with null sectors
        if "OpenBBError" in str(type(e)):
            return pl.LazyFrame({
                "symbol": symbols,
                "sector": [None] * len(symbols)
            })
        else:
            # Re-raise other exceptions
            raise

In [ ]:
df = obb.equity.price.quote(["XLK", "XLE", "AAPL"], provider='yfinance').to_polars()
df

## Test aget_sector_filter

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.analytics.user_table.helpers import aget_sector_filter


(await aget_sector_filter(symbols=["XLU", "XLE", "XLF", "XLC", "KO"])).collect()

# Test aggregate_user_table_data

In [ ]:
from humbldata.portfolio.analytics.user_table.helpers import aggregate_user_table_data


(await aggregate_user_table_data(symbols=["XLU", "XLE", "AAPL"])).collect()

In [ ]:
from humbldata.core.utils.openbb_helpers import aget_latest_price


(await aget_latest_price(symbol=["AAPL", "NVDA", "XLE"])).collect()